<a href="https://colab.research.google.com/github/laura-cramm/CIND-820-Big-Data-Analytics-Project/blob/main/CIND_820_Exploratory_Data_Analysis_(Core_Modules).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b>Conducting Exploratory Data Analysis Using YData Profiling</b>:
###<b>Behavioural Risk Factor Surveillance System Standardized Questionnaire </b>

The Behavioural Risk Factor Surveillance System (BRFSS) consists of a standardized core questionnaire that all states are obligated to conduct, in addition to a series of optional modules and state-added questions. Here, exploratory data analysis will be conducted on the standardized core questionnaire specifically.

In [1]:
!pip install ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 20.9 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=15f25e76d0cd70e86336d52c97bb281142848140a803e7f38771d8f7a40d3a74
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [2]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

In [3]:
#Reading the csv file containing the standardized core BRFSS data and saving
#it to a pandas data frame
brfssCore = pd.read_csv("/content/CoreSurvey.csv")

In [4]:
#The COVIDPOS variable represents the answer to the question: "Has a doctor,
#nurse, or other health professional ever told you that you tested positive for COVID 19"
#A value of 1 represents a "Yes", 2 represents a "No", and 3 represents "Tested positive
#using home test without health professional". All other values indicate missing data.

#Converting the COVIDPOS variable to a Boolean, where 1 and 3 are equivalent to 'True'
#and 2 is equivalent to 'False'
brfssCore['COVIDPOS'] = brfssCore['COVIDPOS'] = np.select([np.logical_or(brfssCore['COVIDPOS'] == 1, brfssCore['COVIDPOS'] == 3), (brfssCore['COVIDPOS'] == 2)],
                                  [True, False], default=None)

In [5]:
#Removing all individuals who never tested positive for COIVD-19 (or had a missing
#value for the COVIDPOS variable) from the data frame.
brfssCore = brfssCore[(brfssCore["COVIDPOS"] == True)]

In [6]:
#The MENTHLTH variable represents the answer to the question:
#"Now thinking about your mental health, which includes stress, depression, and problems with emotions,
#for how many days during the past 30 days was your mental health not good?"
#This key outcome variable will be dichotomized, with 14 or more days
#indicating frequent mental distress.
brfssCore['MENTHLTH'] = np.select([np.logical_or(brfssCore['MENTHLTH'] == 88, brfssCore['MENTHLTH']<14), np.logical_or(brfssCore['MENTHLTH'] == 77, brfssCore['MENTHLTH'] == 99)],
                                  [False, None], default=True)


In [7]:
#Removing the underscore from the _STATE attribute name.
brfssCore.rename(columns={'_STATE': 'STATE'}, inplace=True)

#Removing the underscore from the _BMI5CAT attribute name.
brfssCore.rename(columns={'_BMI5CAT': 'BMI5CAT'}, inplace=True)

#Removing the underscore from the _SMOKER3 attribute name
brfssCore.rename(columns={'_SMOKER3': 'SMOKER3'}, inplace=True)

#Sorting the attribute names alphabetically.
brfssCore = brfssCore.sort_index(axis=1)

#Attributes that start with an underscore were calculated by the CDC staff using other attributes.
#Removing these attributes
brfssCore = brfssCore.drop(brfssCore.loc[:, '_AIDTST4':'_YRSSMOK'].columns, axis=1)
brfssCore = brfssCore.drop(['ALCDAY4', 'WEIGHT2', 'HEIGHT3', 'HTIN4'], axis=1)

In [8]:
#Converting dichotomous categorical variables to Booleans
#1 is YES, 2 is NO, otherwise MISSING
for col in ['CTELENM1', 'PVTRESD1', 'CELPHON1', 'LADULT1', 'CTELNUM1', 'PVTRESD3',
            'CSTATE1', 'RESPSLCT', 'LANDLINE', 'MEDCOST1', 'EXERANY2', 'CVDINFR4',
            'CVDCRHD4', 'CVDSTRK3', 'ASTHMA3', 'ASTHNOW', 'CHCSCNC1', 'CHCOCNC1',
            'CHCCOPD3', 'ADDEPEV3', 'CHCKDNY2', 'HAVARTH4', 'NUMHHOL4', 'VETERAN3',
            'PREGNANT', 'DEAF', 'BLIND', 'DECIDE', 'DIFFWALK', 'DIFFDRES',
            'DIFFALON', 'HADMAM', 'CERVSCRN', 'CRVCLPAP', 'CRVCLHPV', 'HADHYST2',
            'HADSIGM4', 'COLNCNCR', 'VIRCOLO1', 'SMALSTOL', 'STOOLDN2', 'BLDSTFIT',
            'SMOKE100', 'LCSCTSC1', 'LCSSCNCR', 'FLUSHOT7', 'PNEUVAC4', 'HIVTST7',
            'HIVRISK5', 'COVIDSMP', 'INSULIN1', 'FEETSORE', 'SHINGLE2', 'COVIDVA1',
            'COPDCOGH', 'COPDFLEM', 'COPDBRTH', 'COPDBTST', 'CSRVSUM', 'CSRVRTRN',
            'CSRVINST', 'CSRVINSR', 'CSRVDEIN', 'CSRVCLIN', 'CSRVPAIN', 'PSATEST1',
            'CIMEMLOS', 'CDDISCUS', 'CRGVALZD', 'CRGVPER1', 'CRGVHOU1', 'CRGVEXPT',
            'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN', 'ACEDIVRC', 'SDHEMPLY',
            'FOODSTMP', 'SDHBILLS', 'SDHUTILS', 'SDHTRNSP', 'MARJSMOK', 'MARJEAT',
            'MARJVAPE', 'MARJDAB', 'MARJOTHR', 'STOPSMK2', 'MENTCIGS', 'MENTECIG',
            'HEATTBCO', 'ASBIALCH', 'ASBIDRNK', 'ASBIBING', 'ASBIADVC', 'ASBIRDUC',
            'FIREARM5', 'GUNLOAD', 'LOADULK2', 'CASTHDX2', 'CASTHNO2', 'HADSEX',
            'PFPPRVN4', 'RRPHYSM2', 'HPVADVC4', 'DRNKANY6', 'CAREGIV1']:
  brfssCore[col] = np.select([(brfssCore[col] == 1), (brfssCore[col] == 2)],
                                  [True, False], default=None)

#Sex is coded as a dichtomous variable, where 1=Male and 2=Female.
#Converting to a boolean variable where Female=True and Male=False
brfssCore['SEXVAR'] = np.select([(brfssCore['SEXVAR'] == 2), (brfssCore['SEXVAR'] == 1)],
                                  [True, False], default=None)

In [9]:
#Converting dichotomous categorical variables to Booleans. In this case,
#the variables are either True or have a missing value.
#1 is YES, Otherwise is MISSING
for col in ['COLGHOUS', 'STATERE1', 'SAFETIME', 'CELLFON5', 'CADULT1','CCLGHOUS']:
  brfssCore[col] = np.where((brfssCore[col] == 1), True, None)

In [10]:
#The following variables are numeric, but have multiple values that indicate
#a missing observation. For example, the 'HHADULT' variable represents the
#number of adults in the household. A value of 88, 77, or 99 indicates missing data.

#Altering numeric variables so that only a value of 'None' indicates missing data.
brfssCore['HHADULT'] = np.where(np.logical_or(brfssCore['HHADULT'] == 77, brfssCore['HHADULT'] == 88, brfssCore['HHADULT'] == 99), None, brfssCore['HHADULT'])

for col in ['POORHLTH', 'AVEDRNK3', 'DRNK3GE5', 'COPDSMOK', 'MARIJAN1', 'PHYSHLTH']:
  brfssCore[col] = np.select([(brfssCore[col] == 88), np.logical_or(brfssCore[col] == 77, brfssCore[col] == 99)],
                                  [0, None], default=brfssCore[col])


for col in ['SLEPTIM1', 'MAXDRNKS']:
  brfssCore[col] = np.where(np.logical_or(brfssCore[col] == 77, brfssCore[col] == 99), None, brfssCore[col])

brfssCore['CHILDREN'] = np.select([(brfssCore['CHILDREN'] == 88), (brfssCore['CHILDREN'] == 99)],
                                  [0, None], default=brfssCore['CHILDREN'])


brfssCore['LCSFIRST'] = np.where(np.logical_or(brfssCore['LCSFIRST'] == 777, brfssCore['LCSFIRST'] == 888, brfssCore['LCSFIRST'] == 999), None, brfssCore['LCSFIRST'])


for col in ['LCSLAST', 'LCSNUMCG']:
  brfssCore[col] = np.where(np.logical_or(brfssCore[col] == 777, brfssCore[col] == 999), None, brfssCore[col])


brfssCore['CHKHEMO3'] = np.select([np.logical_or(brfssCore['CHKHEMO3'] == 88, brfssCore['CHKHEMO3'] == 98), np.logical_or(brfssCore['CHKHEMO3'] == 77, brfssCore['CHKHEMO3'] == 99)],
                                  [0, None], default=brfssCore['CHKHEMO3'])


brfssCore['CNCRAGE'] = np.where(np.logical_or(brfssCore['CNCRAGE'] == 98, brfssCore['CNCRAGE'] == 99), None, brfssCore['CNCRAGE'])

brfssCore['DROCDY4_'] = np.where((brfssCore['DROCDY4_'] == 900), None, brfssCore['DROCDY4_'])


In [11]:
#The following variables are ordinal, but also have multiple values that indicate
#missing data. Altering these variables so that only a value of 'None' indicates missing data.

for col in ['GENHLTH','COLNTES1', 'SIGMTES1', 'LASTSIG4', 'VCLNTES2', 'STOLTEST', 'SDNATES1',
            'SMOKDAY2', 'USENOW3',  'ECIGNOW2',  'LCSCTWHN',  'TETANUS1', 'PERSDOC3',
            'DIABETE4', 'HOWLONG', 'CRVCLCNC', 'PREDIAB2', 'COVACGET',  'COVIDNU1',
            'COVIDINT', 'CNCRDIFF', 'CSRVCTL2', 'PSATIME1', 'CDHOUSE',   'CDASSIST',
            'CDHELP',   'CDSOCIAL', 'CRGVLNG1',  'CRGVHRS1', 'ACEPUNCH', 'ACEHURT1',
            'ACESWEAR',   'ACETOUCH',  'ACETTHEM', 'ACEHVSEX',  'ACEADSAF',   'ACEADNED',
            'LSATISFY',  'EMTSUPRT',   'SDHISOLT',    'SDHFOOD1',   'SDHSTRE1',   'RRCOGNT2',
            'RRTREAT',  'RRATWRK2']:
            brfssCore[col] = np.where(np.logical_or(brfssCore[col] == 7, brfssCore[col] == 9), None, brfssCore[col])



brfssCore['CHECKUP1'] = np.where(np.logical_or(brfssCore['CHECKUP1'] == 7, brfssCore['CHECKUP1'] == 8, brfssCore['CHECKUP1'] == 9), None, brfssCore['CHECKUP1'])

A = np.logical_or(brfssCore['LASTDEN4'] == 5, brfssCore['LASTDEN4'] == 7)
B = np.logical_or(brfssCore['LASTDEN4'] == 8, brfssCore['LASTDEN4'] == 9)
brfssCore['LASTDEN4'] = np.where(np.logical_or(A, B), None, brfssCore['LASTDEN4'])


for col in ['RMVTETH4', 'PDIABTS1', 'EYEEXAM1', 'DIABEYE1', 'DIABEDU1' ]:
  brfssCore[col] = np.select([(brfssCore[col] == 8), np.logical_or(brfssCore[col] == 7, brfssCore[col] == 9)],
                                  [0, None], default=brfssCore[col])

for col in ['SMOKER3', 'EDUCA']:
  brfssCore[col] = np.where((brfssCore[col] == 9), None, brfssCore[col])

for col in ['INCOME3', 'HPVADSHT', 'LASTSMK2']:
  brfssCore[col] = np.where(np.logical_or(brfssCore[col] == 77, brfssCore[col] == 99), None, brfssCore[col])

brfssCore['RRHCARE4'] = np.where(np.logical_or(brfssCore['RRHCARE4'] == 6, brfssCore['RRHCARE4'] == 7, brfssCore['RRHCARE4'] == 9), None, brfssCore['RRHCARE4'])


brfssCore['_AGEG5YR'] = np.where((brfssCore['_AGEG5YR'] == 14), None, brfssCore['_AGEG5YR'])

brfssCore['_AGE65YR'] = np.where((brfssCore['_AGE65YR'] == 3), None, brfssCore['_AGE65YR'])


#Using the .astype() function to give ordinal variables the 'category' data type
for col in ['GENHLTH','COLNTES1', 'SIGMTES1', 'LASTSIG4', 'VCLNTES2', 'STOLTEST', 'SDNATES1',
            'SMOKDAY2', 'USENOW3',  'ECIGNOW2',  'LCSCTWHN',  'TETANUS1', 'PERSDOC3',
            'DIABETE4', 'HOWLONG', 'CRVCLCNC', 'PREDIAB2', 'COVACGET',  'COVIDNU1',
            'COVIDINT', 'CNCRDIFF', 'CSRVCTL2', 'PSATIME1', 'CDHOUSE',   'CDASSIST',
            'CDHELP',   'CDSOCIAL', 'CRGVLNG1',  'CRGVHRS1', 'ACEPUNCH', 'ACEHURT1',
            'ACESWEAR',   'ACETOUCH',  'ACETTHEM', 'ACEHVSEX',  'ACEADSAF',   'ACEADNED',
            'LSATISFY',  'EMTSUPRT',   'SDHISOLT',    'SDHFOOD1',   'SDHSTRE1',   'RRCOGNT2',
            'RRTREAT',  'RRATWRK2', 'CHECKUP1', 'LASTDEN4', 'RMVTETH4', 'PDIABTS1', 'EYEEXAM1', 'DIABEYE1', 'DIABEDU1',
            'EDUCA', 'INCOME3', 'HPVADSHT', 'LASTSMK2', 'RRHCARE4', '_AGEG5YR', '_AGE65YR',  'BMI5CAT', 'SMOKER3' ]:
    brfssCore[col] = brfssCore[col].astype('category')

In [12]:
#These variables are nominal; dummy variables will be created using
#one-hot encoding.
for col in ['STATE', 'FMONTH','IMONTH','SEXVAR', 'QSTVER','QSTLANG','MSCODE']:
  brfssCore = pd.concat([brfssCore, pd.get_dummies(brfssCore[col], prefix=col)], axis=1)

#First altering these variables so that only a value of 'None' indicates missing data.
brfssCore['IDAY'] = np.where((brfssCore['IDAY'] == 0), None, brfssCore['IDAY'])

brfssCore['LANDSEX1'] = np.where(np.logical_or(brfssCore['LANDSEX1'] == 0, brfssCore['LANDSEX1'] == 7, brfssCore['LANDSEX1'] == 9), None, brfssCore['LANDSEX1'])

for col in ['CELLSEX1', 'RENTHOM1', 'COLNSIGM', 'DIABTYPE', 'CSRVTRT3',
            'PCPSARS2', 'PSASUGST', 'PCSTALK1', 'RCSXBRTH', 'RCSRLTN2','BIRTHSEX',
            'SOMALE', 'SOFEMALE', 'TRNSGNDR']:
            brfssCore[col] = np.where(np.logical_or(brfssCore[col] == 7, brfssCore[col] == 9), None, brfssCore[col])

brfssCore['PRIMINSR'] = np.select([(brfssCore['PRIMINSR'] == 88), np.logical_or(brfssCore['PRIMINSR'] == 77, brfssCore['PRIMINSR'] == 99)],
                                  [0, None], default=brfssCore['PRIMINSR'])


for col in ['MARITAL', 'EMPLOY1', 'RCSGEND1']:
  brfssCore[col] = np.where((brfssCore[col] == 9), None, brfssCore[col])


for col in ['COVIDPRM','IMFVPLA3','CNCRTYP2','CSRVDOC1', 'CRGVREL4',
            'CRGVPRB3', 'TYPCNTR9','BRTHCNT4','WHEREGET', 'NOBCUSE8',
            'BCPREFER','RRCLASS3']:
            brfssCore[col] = np.where(np.logical_or(brfssCore[col] == 77, brfssCore[col] == 99), None, brfssCore[col])

#Generating dummy variables
for col in ['IDAY', 'LANDSEX1', 'CELLSEX1', 'RENTHOM1', 'COLNSIGM', 'DIABTYPE', 'CSRVTRT3',
            'PCPSARS2', 'PSASUGST', 'PCSTALK1', 'RCSXBRTH', 'RCSRLTN2','BIRTHSEX',
            'SOMALE', 'SOFEMALE', 'TRNSGNDR', 'PRIMINSR',  'MARITAL', 'EMPLOY1', 'RCSGEND1', 'COVIDPRM','IMFVPLA3','CNCRTYP2','CSRVDOC1', 'CRGVREL4',
            'CRGVPRB3', 'TYPCNTR9','BRTHCNT4','WHEREGET', 'NOBCUSE8',
            'BCPREFER','RRCLASS3']:
  brfssCore = pd.concat([brfssCore, pd.get_dummies(brfssCore[col], prefix=col)], axis=1)

In [13]:
from datetime import datetime

In [14]:
#These variables represent dates. Processing is required to convert these
#variables to date objects.

for col in ['FLSHTMY3', 'HIVTSTD3','COVIDFS1', 'COVIDSE1']:
#Converting to a string object
  brfssCore[col] = brfssCore[col].astype('string')
#Removing decimal places
  brfssCore[col] = brfssCore[col].str.replace('.0', '')
#If string is null, replacing with '-1'
  brfssCore[col] = np.where((brfssCore[col].isnull()), '-1', brfssCore[col])

In [15]:
#Adding preceding zero if the month is only one digit (Ex. 1 to 01)
for col in ['FLSHTMY3', 'HIVTSTD3','COVIDFS1', 'COVIDSE1']:
  for i in range(len(brfssCore[col])):
    if len(brfssCore.loc[brfssCore.index[i], col]) == 5:
      brfssCore.loc[brfssCore.index[i], col] = '0' + brfssCore.loc[brfssCore.index[i], col]

In [16]:
#Some additional values represent a null value; setting to None
for col in ['FLSHTMY3', 'HIVTSTD3','COVIDFS1', 'COVIDSE1']:
  for i in range(len(brfssCore[col])):
    if np.logical_or(brfssCore.loc[brfssCore.index[i], col].endswith('7777'),
                     brfssCore.loc[brfssCore.index[i], col].endswith('9999'), ):

      brfssCore.loc[brfssCore.index[i], col] = None

In [17]:
#Converting all None values to '052024' (which doesn't otherwise appear in dataset)
for col in ['FLSHTMY3', 'HIVTSTD3','COVIDFS1', 'COVIDSE1']:
  for i in range(len(brfssCore[col])):
    if brfssCore.loc[brfssCore.index[i], col] == '-1':
      brfssCore.loc[brfssCore.index[i], col] = None
    if brfssCore.loc[brfssCore.index[i], col] == None:
      brfssCore.loc[brfssCore.index[i], col] = "052024"

In [18]:
#Converting strings to datetime objects using the strptime() function
for col in ['FLSHTMY3', 'HIVTSTD3','COVIDFS1', 'COVIDSE1']:
  for i in range(len(brfssCore[col])):
    #If month is missing, only extract year
    if np.logical_or(brfssCore.loc[brfssCore.index[i], col].startswith('77'),
                     brfssCore.loc[brfssCore.index[i], col].startswith('99'), ):
      brfssCore.loc[brfssCore.index[i], col] =brfssCore.loc[brfssCore.index[i], col][2:]
      brfssCore.loc[brfssCore.index[i], col] = datetime.strptime(brfssCore.loc[brfssCore.index[i], col], '%Y')
    else:
      #Otherwise format is mmYYYY
       brfssCore.loc[brfssCore.index[i], col] = datetime.strptime(brfssCore.loc[brfssCore.index[i], col], '%m%Y')

In [19]:
#Converting observations that were originally missing back to None values
#(the strptime() function cannot handle null values, so a temporary placeholder
#was necessary)
for col in ['FLSHTMY3', 'HIVTSTD3','COVIDFS1', 'COVIDSE1']:
  for i in range(len(brfssCore[col])):
    if brfssCore.loc[brfssCore.index[i], col].year ==2024:
      brfssCore.loc[brfssCore.index[i], col] = None

In [20]:
#'IDATE' (interview date) also needs to be converted to a date object
#but it has a different format than the preceding date variables (mmddYYY vs. mmYYYY)

#Converting to a string object
brfssCore['IDATE'] = brfssCore['IDATE'].astype('string')
#Removing decimal places
brfssCore['IDATE'] = brfssCore['IDATE'].str.replace('.0', '')

In [21]:
#Adding preceding zero if the day is only one digit (Ex. 1 to 01)
for i in range(len(brfssCore['IDATE'])):
  if len(brfssCore.loc[brfssCore.index[i], 'IDATE']) == 7:
    brfssCore.loc[brfssCore.index[i], 'IDATE'] = '0' + brfssCore.loc[brfssCore.index[i], 'IDATE']

In [22]:
#Converting strings to datetime objects
for i in range(len(brfssCore['IDATE'])):
    brfssCore.loc[brfssCore.index[i], 'INTERVIEWDATE'] = datetime.strptime(brfssCore.loc[brfssCore.index[i], 'IDATE'], '%m%d%Y')

<ipython-input-22-83bef59ba8e7>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  brfssCore.loc[brfssCore.index[i], 'INTERVIEWDATE'] = datetime.strptime(brfssCore.loc[brfssCore.index[i], 'IDATE'], '%m%d%Y')


In [23]:
brfssCore = brfssCore.drop(['IDATE'], axis=1)

In [27]:
#Using the ydata_profiling library to generate an exploratory data analysis report
edaCore = ProfileReport(brfssCore, title="Exploratory Data Analysis", minimal=True)

In [28]:
#Saving this exploratory data analysis report to an .html file.
edaCore.to_file("exploratory_data_analysis_core_modules.html")

/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:363: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]